In this notebook we explore how well the algorithm is capable of recovering the following dependencies:
- Linear combination of 3 variables and bias with standard-normally distributed coefficients
  
  We can see that in 6 cases out of 10 the formulas are recovered exactly, while in other 5 cases there is a strong similarity (the degree of one variable is a small number, and the loss is compensated by the bias term).


- Linear combination of 3 squared variables and bias with standard-normally distributed coefficients

    We can clearly see that in 9 cases out of 10 the formulas are recovered exactly.


- Linear combination of 3 variables with powers uniformly distributed over {1, 2 ,3 ,4, 5} and standard-normally distributed coefficients

    We can see that in 6 cases out of 10 the formulas are recovered exactly, while in other 3 cases there is no similarity (the degree of one variable is a small number, and the loss is compensated by the bias term).

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from torch.nn import MSELoss
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
import time
from IPython.display import display, Math, Latex

import Formula
import functions
from functions import *
import importlib
from hessian import hessian
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

In [2]:
importlib.reload(Formula)
importlib.reload(functions)

<module 'functions' from '/home/zybinmikhail/Documents/personal github projects/LearningFormulas/functions.py'>

In [3]:
def print_sharps():
    print("\n################################################################################")

In [4]:
def print_results(symbolic_mse_list, number_of_tested_formulas, recovery_threshold=1e-5):
    print_sharps()
    print("MSEs between parameters:")
    print(symbolic_mse_list)
    symbolic_mse_list = np.array(symbolic_mse_list)
    number_of_small_errors = (symbolic_mse_list < recovery_threshold).sum()
    print(f"For {number_of_small_errors} formulas out of {number_of_tested_formulas} ", end="") 
    print(f"the error is less than {recovery_threshold}.")

In [5]:
def print_time(cnt, cnt_iteration):
    time_from_start = time.perf_counter() - cnt
    time_iteration = time.perf_counter() - cnt_iteration
    min_from_start, sec_from_start = divmod(time_from_start, 60)
    min_iteration, sec_iteration = divmod(time_iteration, 60)    
    print(f"{remove_zero_minutes(min_from_start)}{sec_from_start :.0f} seconds passed from the start, ", end="")
    print(f"the iteration took {remove_zero_minutes(min_iteration)}{sec_iteration :.0f} seconds")

In [6]:
def get_params(regressor):
    lambdas = [regressor.get_lambda(i).item() for i in range(3)]
    powers = [regressor.get_power(i).item() for i in range(3)]
    bias_term = [regressor.last_subformula.lambda_0.item()]
    obtained_params = np.array(lambdas + powers + bias_term)
    return obtained_params

In [7]:
def generate_data(n_variables=3, m_samples=1000, min_power=1, max_power=2):
    X = torch.rand(m_samples, n_variables)
    b = torch.randn(1)
    coeffs = torch.randn((n_variables, 1))
    powers = torch.randint(min_power, max_power, (1, n_variables))
    y = X**powers @ coeffs + b
    true_params = np.array(coeffs.view(-1,).tolist() + powers.view(-1,).tolist() + [b.item()])
    return coeffs, powers, b, X, y, true_params

In [8]:
def print_ground_truth(coeffs, powers, b):
    formula = []
    powers = powers.view(-1,)
    for i in range(len(coeffs)):
        new_term = [round(coeffs[i].item(), 3), "x_{", i + 1, "}^{", round(powers[i].item(), 3), "}"]
        if new_term[0] > 0 and i > 0:
            formula.append('+')
        formula.extend(list(map(str, new_term)))
    if b > 0:
        formula.append("+")
    formula.append(str(round(b.item(), 3)))
    PrintFormula("".join(formula))

In [9]:
def explore(n_variables=3, m_samples=1000, min_power=1, max_power=1, number_of_tested_formulas=10, recovery_threshold=1e-5):
    cnt = time.perf_counter()
    symbolic_mse_list = []

    for i in range(number_of_tested_formulas):
        print(f"\n\n----------------------------Exploring new formula #{i + 1}----------------------------")
        coeffs, powers, b, X, y, true_params = generate_data(n_variables, m_samples, min_power, max_power + 1)
        cnt_iteration = time.perf_counter()
        regressor, _ = Formula.LearnFormula(X, y, optimizer_for_formula=torch.optim.Rprop, n_init=10)
        print_time(cnt, cnt_iteration)
        print("ground truth and obtained formula")
        print_ground_truth(coeffs, powers, b)
        PrintFormula(regressor)

        obtained_params = get_params(regressor)
        symbolic_mse = mean_squared_error(true_params, obtained_params)
        symbolic_mse_list.append(symbolic_mse)
        print(f"MSE between formula parameters is {symbolic_mse}")
        if symbolic_mse < recovery_threshold:
            print("EXACT RECOVERY")
        else:
            print("FAILURE")

    print_results(symbolic_mse_list, number_of_tested_formulas)

## Linear combination of 3 variables and bias with standard-normally distributed coefficients

In [10]:
explore()



----------------------------Exploring new formula #1----------------------------
  Initialization #1
  Finished run #1, loss 1.9545899121453658e-08, best loss 1.9545899121453658e-08
loss is smaller than 0.001, terminating learning process
3 seconds passed from the start, the iteration took 3 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

<IPython.core.display.Math object>

MSE between formula parameters is 1.0063899250464407e-06
EXACT RECOVERY


----------------------------Exploring new formula #2----------------------------
  Initialization #1
  Finished run #1, loss 2.3432189522054614e-09, best loss 2.3432189522054614e-09
loss is smaller than 0.001, terminating learning process
4 seconds passed from the start, the iteration took 2 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

MSE between formula parameters is 2.2995982725180853e-07
EXACT RECOVERY


----------------------------Exploring new formula #10----------------------------
  Initialization #1
  Finished run #1, loss 0.0022249321918934584, best loss 0.0022249321918934584
  Initialization #2
  Finished run #2, loss 0.002683191327378154, best loss 0.0022249321918934584
  Initialization #3
  Finished run #3, loss 0.0011217775754630566, best loss 0.0011217775754630566
  Initialization #4
  Finished run #4, loss 5.520704449146763e-10, best loss 5.520704449146763e-10
loss is smaller than 0.001, terminating learning process
1 minutes 20 seconds passed from the start, the iteration took 8 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

<IPython.core.display.Math object>

MSE between formula parameters is 2.7258411502264053e-06
EXACT RECOVERY

################################################################################
MSEs between parameters:
[1.0063899250464407e-06, 3.308736343005896e-07, 5.875073967003581e-08, 0.40518855792418684, 4.3907026754068976e-07, 0.2654291738345941, 0.0004490945575166906, 0.41407103044771354, 2.2995982725180853e-07, 2.7258411502264053e-06]
For 6 formulas out of 10 the error is less than 1e-05.


## Linear combination of 3 squared variables and bias with standard-normally distributed coefficients

In [11]:
explore(min_power=2, max_power=2)



----------------------------Exploring new formula #1----------------------------
  Initialization #1
  Finished run #1, loss 0.19504092633724213, best loss 0.19504092633724213
  Initialization #2
  Finished run #2, loss 0.007013747934252024, best loss 0.007013747934252024
  Initialization #3
  Finished run #3, loss 0.009375261142849922, best loss 0.007013747934252024
  Initialization #4
  Finished run #4, loss 0.009800040163099766, best loss 0.007013747934252024
  Initialization #5
  Finished run #5, loss 1.8762023185070476e-12, best loss 1.8762023185070476e-12
loss is smaller than 0.001, terminating learning process
15 seconds passed from the start, the iteration took 15 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

<IPython.core.display.Math object>

MSE between formula parameters is 6.69699216717033e-12
EXACT RECOVERY


----------------------------Exploring new formula #2----------------------------
  Initialization #1
  Finished run #1, loss 3.5127137743701242e-12, best loss 3.5127137743701242e-12
loss is smaller than 0.001, terminating learning process
16 seconds passed from the start, the iteration took 1 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

MSE between formula parameters is 7.782828465030533e-08
EXACT RECOVERY


----------------------------Exploring new formula #10----------------------------
  Initialization #1
  Finished run #1, loss 7.292567591465904e-12, best loss 7.292567591465904e-12
loss is smaller than 0.001, terminating learning process
2 minutes 55 seconds passed from the start, the iteration took 6 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

<IPython.core.display.Math object>

MSE between formula parameters is 5.295644557788495e-10
EXACT RECOVERY

################################################################################
MSEs between parameters:
[6.69699216717033e-12, 7.412026548081485e-11, 3.558649565503304e-10, 4.6123866630815817e-10, 3.520428805612953e-09, 3.1134510842483516e-10, 6.36695601003672e-09, 2.306655452500273, 7.782828465030533e-08, 5.295644557788495e-10]
For 9 formulas out of 10 the error is less than 1e-05.


## Linear combination of 3 variables with powers uniformly distributed over {1, 2 ,3 ,4, 5} and standard-normally distributed coefficients

In [12]:
explore(max_power=5)



----------------------------Exploring new formula #1----------------------------
  Initialization #1
  Finished run #1, loss 0.2498483806848526, best loss 0.2498483806848526
  Initialization #2
  Finished run #2, loss 0.24947260320186615, best loss 0.24947260320186615
  Initialization #3
  Finished run #3, loss 4.875495623543613e-13, best loss 4.875495623543613e-13
loss is smaller than 0.001, terminating learning process
14 seconds passed from the start, the iteration took 14 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

<IPython.core.display.Math object>

MSE between formula parameters is 2.6391587328232295e-13
EXACT RECOVERY


----------------------------Exploring new formula #2----------------------------
  Initialization #1
  Finished run #1, loss 0.00034576840698719025, best loss 0.00034576840698719025
loss is smaller than 0.001, terminating learning process
16 seconds passed from the start, the iteration took 2 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

MSE between formula parameters is 0.7749553351816612
FAILURE


----------------------------Exploring new formula #10----------------------------
  Initialization #1
  Finished run #1, loss 0.00399942509829998, best loss 0.00399942509829998
  Initialization #2
  Finished run #2, loss 3.6368278180654023e-12, best loss 3.6368278180654023e-12
loss is smaller than 0.001, terminating learning process
53 seconds passed from the start, the iteration took 4 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

<IPython.core.display.Math object>

MSE between formula parameters is 1.8920433473925057e-10
EXACT RECOVERY

################################################################################
MSEs between parameters:
[2.6391587328232295e-13, 0.31358897096567057, 1.9162608008319435e-11, 2.532678828564323e-11, 5.2594433253005946e-09, 2.27516452866415, 1.6838589950956314e-08, 3.337249927827354, 0.7749553351816612, 1.8920433473925057e-10]
For 6 formulas out of 10 the error is less than 1e-05.
